In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Api Information")
 
# Ready the files for upload to OpenAI
file_paths = ["api-info.txt"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [4]:
assistant = client.beta.assistants.create(
  name="Api Url Assistant",
  instructions="""
  You provide the info about the api and the api url to call for the users querry.
  You should only provide the api info about the users prompt, should not provide anything else.
  The response should contain only the python code for calling the api and then storing the response in a variable called responses.
  No need of any import statements as well as markdown formatting.
  """,
  model="gpt-4o",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

The commented stuff below is for attaching some new files to the vector store

In [6]:
# # Upload the user provided file to OpenAI
# message_file = client.files.create(
#   file=open("api-info.txt", "rb"), purpose="assistants"
# )
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "get me a cat fact",
      # Attach the new file to the message.
      # "attachments": [
      #   { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      # ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

None


In [7]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
# print("\n".join(citations))

responses = requests.get("https://catfact.ninja/fact")
